In [1]:
using DataFrames
using Chemfiles

home = "/home/german/labo/18/egfr/model/"

In [2]:
function res_3_to_1(in_3::AbstractString)
    
    res = lowercase(in_3)
    out_1 = AbstractString
    
    if res == "ala"
        out_1 = "A"
    elseif res == "arg"
        out_1 = "R"
    elseif res == "asn"
        out_1 = "N"
    elseif res == "asp"
        out_1 = "D"
    elseif res == "cys"
        out_1 = "C"
    elseif res == "gln"
        out_1 = "Q"
    elseif res == "glu"
        out_1 = "E"
    elseif res == "gly"
        out_1 = "G"
    elseif res == "his"
        out_1 = "H"
    elseif res == "ile"
        out_1 = "I"
    elseif res == "leu"
        out_1 = "L"
    elseif res == "lys"
        out_1 = "K"
    elseif res == "met"
        out_1 = "M"
    elseif res == "phe"
        out_1 = "F"
    elseif res == "pro"
        out_1 = "P"
    elseif res == "ser"
        out_1 = "S"
    elseif res == "thr"
        out_1 = "T"
    elseif res == "trp"
        out_1 = "W"
    elseif res == "tyr"
        out_1 = "Y"
    elseif res == "val"
        out_1 = "V"
    else
        error("Unknown aminoacid: ", in_3)
    end
    
    return out_1
end

function res_3_to_1(in_vec::Array{String, 1})

    out_vec = Array{String, 1}(length(in_vec))
    i = 0
    for each in in_vec
        i+=1
        try 
            out_vec[i] = res_3_to_1(each)
        catch e
            error("Malformed array list.\nCaught error: $e")
        end
    end
    
    return out_vec
end

res_3_to_1 (generic function with 2 methods)

In [3]:
function seq_from_top(in_top::Chemfiles.Topology)
    # Get 1 letter code amino acid list from a topology
    aa = convert(Int64, count_residues(in_pdb_top))
    ids = Array{Int64, 1}(aa)

    for i=1:aa
        ids[i] = id(Residue(in_pdb_top, i-1))
    end
    ids_sorted_indices = sortperm(ids);

    seq_3 = Array{String, 1}(aa)
    i = 0
    for each in ids_sorted_indices
        i+=1
        seq_3[i] = name((Residue(in_pdb_top, each-1)))
    end
    seq_1 = res_3_to_1(seq_3);
    
    return seq_1
end

seq_from_top (generic function with 1 method)

In [4]:
each = "1XKK_A"
in_pdb_trj = Trajectory(string(home, each, "/", each, ".pdb"))
in_pdb_frm = read(in_pdb_trj)
in_pdb_top = Topology(in_pdb_frm)
seq_from_top(in_pdb_top)

Chemfiles.Topology(Ptr{Chemfiles.lib.CHFL_TOPOLOGY} @0x00000000036b4a50)

289-element Array{String,1}:
 "A"
 "L"
 "L"
 "R"
 "I"
 "L"
 "K"
 "E"
 "T"
 "E"
 "F"
 "K"
 "K"
 ⋮  
 "M"
 "D"
 "E"
 "V"
 "V"
 "D"
 "A"
 "D"
 "E"
 "Y"
 "L"
 "I"